In [93]:
import cvxpy as cp
import torch 
from cvxpylayers.torch import CvxpyLayer
from numpy.random import rand
import numpy as np
import cProfile
from memory_profiler import profile

In [87]:
# starting w/ a simple LP

D = 50  # variable dimension
M = 50  # no of equality constraints
N = 60; # no of inequality constraints

x̂ = rand(D) # solution

_c = rand(D) # objective coeffs

_A = rand(M, D) # equality part
_b = np.matmul(_A, x̂)

_G = rand(N, D) # inequality part
_h = np.matmul(_G, x̂) + rand(N);

In [88]:
x = cp.Variable(D)
c = cp.Parameter(D)
A = cp.Parameter((M, D))
b = cp.Parameter(M)
G = cp.Parameter((N, D))
h = cp.Parameter(N)
constraints = [A*x == b, G*x <= h]
objective = cp.Minimize(c @ x)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

/home/akshay/.pyenv/versions/procol/lib/python3.6/site-packages/cvxpy/expressions/expression.py:503: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/home/akshay/.pyenv/versions/procol/lib/python3.6/site-packages/cvxpy/expressions/expression.py:503: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [89]:
cvxpylayer = CvxpyLayer(problem, parameters=[c,A,b,G,h], variables=[x])
c_t = torch.Tensor(_c)
A_t = torch.Tensor(_A)
b_t = torch.Tensor(_b)
G_t = torch.Tensor(_G)
h_t = torch.Tensor(_h)

c_t.requires_grad=True
A_t.requires_grad=True
b_t.requires_grad=True
G_t.requires_grad=True
h_t.requires_grad=True

# solve the problem
solution, = cvxpylayer(c_t, A_t, b_t, G_t, h_t)

/home/akshay/.pyenv/versions/procol/lib/python3.6/site-packages/diffcp/cone_program.py:263: UserWarning: Solved/Inaccurate.
  warnings.warn("Solved/Inaccurate.")


In [90]:
# compute the gradient of the sum of the solution with respect to A, b
cProfile.run('solution.sum().backward()')

         14 function calls in 0.030 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.030    0.030 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:20(_make_grads)
        1    0.000    0.000    0.029    0.029 __init__.py:44(backward)
        1    0.000    0.000    0.029    0.029 tensor.py:167(backward)
        1    0.000    0.000    0.030    0.030 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method ones_like}
        1    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'numel' of 'torch._C._Tenso

In [105]:
@profile 
def myGrad(solution):
    solution.sum().backward()

In [106]:
myGrad(solution)

ERROR: Could not find file <ipython-input-105-975679b00c61>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
